---
title: "나의 뉴욕 탈출기"
date: "2025-03-12"
author: "구경민"
categories: [code, analysis]
format: 
    html:
        toc: true
        toc-depth: 3
        theme: cosmo
        code-fold:  true
        search: true    
        number-sections: true
        highlight-style: a11y
# image: 
jupyter: python3
---


## Contents

1.  문제 정의
2.  데이터 로드와 데이터 정보 확인
3.  \[Q1\] 출발 지연이 빈번하게 발생하는 월(month)은 언제인가?
4.  \[Q2\] 출발 지연이 빈번하게 발생하는 시간(hour)은 언제인가?
5.  \[Q3\] 출발 지연이 평균적으로 긴 항공사는 무엇인가?
6.  결론

## 문제 정의

문득, 뉴욕이 싫어져서 떠나기로 했다. 비싼 물가, 귀 떨어질 것 같은 소음, 그리고 꽉 막힌 교통환경 등으로 뉴욕이 아닌 다른 곳으로 가고 싶어졌다. 

일단, 상황이 복잡한건 싫으니 출발 지연이 최대한 적은 방향으로 이 복잡한 도시를 탈출하려고 한다. 

마침 2013년 뉴욕주 내 공항 정보가 담긴 [nycflight](https://www.openintro.org/data/index.php?data=nycflights) 데이터가 있으므로, 이 데이터를 이용하여 비행 지연이 빈번히 일어나는 시간과 비행 지연을 밥 먹듯이 하는 비행사를 찾아 최소한의 탈출계획을 세워보기로 하였다.

**Summary**

-   출발 지연이 빈번하게 발생하는 월(month)은 언제인가?

-   출발 지연이 빈번하게 발생하는 시간(hour)은 언제인가?

-   출발 지연이 평균적으로 긴 항공사는 무엇인가?

## 데이터 로드와 데이터 정보 확인


In [ ]:
import os
import pandas as pd
import numpy as np
# from nycflights13 import flights

import warnings
warnings.filterwarnings('ignore')

nycfl = pd.read_csv('./nycflights.csv')
print(nycfl.info())
print(nycfl.describe())

<!-- It needs to write the information and summary statistics of dataset -->

## \[Q1\] 출발 지연이 빈번하게 발생하는 월(month)은 언제인가?

먼저 출발 지연이 자주 발생하는 월(month)을 찾아서 그 달만큼은 피해보기로 하였다.


In [ ]:
positive_dep_delay = nycfl.loc[nycfl['dep_delay']>0]
# positive_dep_delay.loc[:, 'month'].mode()

months = [m for m in range(1,13)]
cnt_delays_dep = list()

for m in months:
    dep_delays = len(positive_dep_delay[positive_dep_delay['month'] == m])
    cnt_delays_dep.append(dep_delays)
    print(f'{m} | {dep_delays}')
print('----------------')

max_idx = np.argmax(np.array(cnt_delays_dep)) 
print(f'출발 지연이 가장 빈번히 발생한 달: {max_idx+1} | 발생 횟수: {cnt_delays_dep[max_idx]}')

먼저, 변수 `positive_dep_delay`는 `dep_delay`가 양수인 경우(지연이 발생한 시간)만 filtering한 record만 저장하였다.

이를 이용하여 `positive_dep_delay.loc[:, 'month'].mode()`를 통해 출발 지연이 가장 비번히 발생한 달을 빠르게 찾을 수 있지만, 그 횟수까지 알고 싶으므로 월별 출발 지연 횟수를 확인해보았다.

`for m in months:`부터 `print(f'{m} | {dep_delays}')`까지의 코드는 월별 출발 지연 횟수를 counting하여 `cnt_delays_dep`에 저장하고, 해당 월의 출발 지연 횟수를 조회하는 것을 의미한다.

마지막으로 저장한 리스트에서 출발 지연이 가장 많이 발생한 월과 그 횟수를 보여준다.

해당 결과를 바탕으로 아래와 같이 알 수 있었다.

**Summary**

-   12월에 1373번으로 출발 지연이 가장 많이 발생하였다.
-   하지만 조회 결과에 따르면 7월에 1334번으로 가장 많이 발생한 횟수와 39번으로 큰 차이가 없다.
-   **결론적으로, 7월과 12월에 떠나는 것을 피한다.**

## \[Q2\] 출발 지연이 빈번하게 발생하는 시간(hour)은 언제인가?

\[section 1\]에서 만든 `positive_dep_delay`를 이용하여 빈번하게 발생하는 출발 지연 시간(hour)를 알아보자.


In [ ]:
def three_to_four(x):
    if len(x)==3:
        
        x_li = list(x)
        new_li = [None]*4
        new_li[0] = '0'
        
        for c, x in enumerate(x_li):
            new_li[c+1] = x
        
        return ''.join(new_li)
    else:
        return x

positive_dep_delay['dep_time'] = positive_dep_delay['dep_time'].astype(str)
positive_dep_delay['dep_time'] = positive_dep_delay['dep_time'].apply(three_to_four)

positive_dep_delay['dep_time_hour'], positive_dep_delay['dep_time_min'] = positive_dep_delay['dep_time'].apply(lambda x: x[:2]), positive_dep_delay['dep_time'].apply(lambda x: x[2:])

print(positive_dep_delay['dep_time_hour'].mode())

\[section 2\]의 각 속성의 데이터 유형에 따르면 출발 시간인 `dep_time`은 int로 앞의 두자리는 시간, 나머지는 분을 의미하는 것으로 보인다. 이를 떼어내서 시간만 추출할 수 있도록 해보자.

먼저, `dep_time`의 각 element 길이는 3\~4로 파악이 된다. 

즉, 시간은 24시간을 기준으로 관측이 된것으로, 오전 시간 앞에 0을 붙혀서 모든 element의 길이를 4자리로 일관성있게 바꿔줘야 한다. 

이를 처리하기 위해 `dep_time`의 자료형을 `object`형으로 바꿔준다.

함수 `three_to_four`는 3자리의 element를 4자리로 바꾸기 위해 길이가 4인 리스트를 선언하고 0번 index의 element를 "0" 으로 지정하게 한다. 

다음으로 길이가 3인 문자열의 각 string element를 list의 element로 하나씩 추가시키고 이를 합치고 반환한다.

다음으로 `lambda`함수를 이용해서 처리하여 얻은 `dep_time`의 앞에서 2자리는 시간, 나머지는 분으로 분리하여 각 관련 파생변수(`dep_time_hour`와 `dep_time_min`)로 만들어서 저장한다.

마지막으로, 지연이 빈번하게 발생한 시간을 출력하게 한다. 이에 대한 결과를 바탕으로 아래와 같이 알 수 있었다.

**Summary**

- 출발 지연이 17시에 출발하는 항공편에서 빈번하게 발생하므로, 17시 출발하는 비행기는 피한다.

## \[Q3\] 출발 지연이 평균적으로 긴 항공사는 무엇인가?

마지막으로, 출발 지연이 평균적으로 긴 항공사를 파악하여 1순위로 파악해야 하는 항공사를 알아보자.


In [ ]:
def find_dest(origin,dest):
    positive_dep_delay = nycfl[nycfl['dep_delay'] > 0]
    positive_ori_dest = positive_dep_delay.loc[(nycfl['origin']==origin) & (nycfl['dest']==dest)]
    
    if len(positive_ori_dest) == 0:
        return None, None
    else:
        mean_delay_dep = positive_ori_dest.groupby('carrier')['dep_delay'].mean()
        return  mean_delay_dep.idxmax(), mean_delay_dep.max() 

origin_pl = set(nycfl['origin'].to_list())
dest_pl = set(nycfl['dest'].to_list())
ori_des = dict()

for o in origin_pl:
    max_mean_carrier = 0
    max_dst, saved_max_mean_carrier = str(), str()
    
    for d in dest_pl:
        max_mean_delay_carrier, max_mean_delay = find_dest(o, d)
        
        if max_mean_delay is None:
            continue
        
        if max_mean_delay > max_mean_carrier:
            max_dst = d 
            max_mean_carrier =  max_mean_delay
            saved_max_mean_carrier = max_mean_delay_carrier 

    ori_des[o] = [max_dst, max_mean_carrier, saved_max_mean_carrier]
ori_des

먼저, 뉴욕주 내의 공항은 여러 곳이 있다. 

수월한 분석을 위해 출발지인 `origin_pl`와 도착지 정보를 담은 `dest_pl`을 선언한다. 

다음으로 `for o in origin_pl:` 부터 `for d in dest_pl:`는 출발지 별 최대 평균 출발 지연시간을 갖는 항공사, 도착지를 찾기 위한 loop이다. 

먼저, `max_mean_carrier`, `max_dst`, 그리고 `saved_max_mean_carrier`는 각 최대 출발 지연 평균, 도착지, 그리고 항공사를 초기화를 한다. 

이는 각 최대 평균 중 최대값을 찾으면 관련 값들을 갱신하기 위해 선언하였다.

출발지에 대한 각 도착지 간 최대 평균을 구하기 위해 `find_dest` 함수를 설계하였다. 

이 함수는 최대 지연 평균 시간을 가지는 항공사와 그 목적지를 찾기 위해 설계된 함수로 다음과 같이 작동한다.

> 1. 이전과 같이 출발 지연이 있는 record를 뽑는다.
> 2. filtering한 record 데이터에서 해당 출발지와 도착지만 해당하는 데이터를 다시 filtering한다.
> 3. 만약 filtering한 데이터가 없으면 None을 반환하고
> 4. 존재한다면 항공사를 grouping하여 각 출발 지연 평균 시간을 계산한다.
> 5. 최대 출발 지연 시간을 가지는 항공사와 그 출발 지연 평균 시간을 반환한다.

위의 방법으로 출발지 별 최대 출발 지연 평균 시간, 항공사를 뽑아내고, 해당 최대 평균 시간이 다른 도착지들 보다 높으면 정보를 갱신한다.

위의 방법을 통해 아래와 같이 알 수 있었다.

**Summary**
- 뉴욕 라과디아 공항(LGA)에서 Endeavor 항공(9E)을 통해 버지니아주 샌드스톤에 위치한 리치먼드 공항(RIC)으로 갈때 출발 지연 시간이 평균적으로 많이 걸린다.
- John F. Kennedy 공항(JFK)에서 하와이안 항공(HA)을 통해 하와이에 위치한 다니엘 K. 이노우에 공항(HNL)으로 갈떄 출발 지연 시간이 평균적으로 많이 걸린다.
- 뉴어크 리버티 공항(EWR)에서 유나이티드 항공(UA)을 통해 세인트루이스 램버트 공항(STL)으로 갈때 출발 지연 시간이 평균적으로 많이 걸린다.
- __이중 John F. Kennedy 공항(JFK)에서 하와이안 항공을 통해 하와이로 갈때 평균 지연시간이 제일 많이 걸리므로 하와이로 갈때는 이 노선을 피해야 할 것이다.__

## 결론
지금까지의 분석을 통해 다음과 같은 결론을 낼 수 있었다.
- 